In [1]:
import pandas as pd
data_path = "../tuntun_data/Daily93Tuntun_2010-2021.csv"  # Replace with your file path
df = pd.read_csv(data_path)

In [8]:
df

,date,open,high,low,close,volume,tic,day
50437,2020-01-02,14550.00,14650.00,14025.00,14025.00,799000,AALI,6591
50438,2020-01-02,84.22,85.81,83.43,83.43,7980487,ABBA,6591
50439,2020-01-02,7075.00,7075.00,6925.00,6925.00,1700,ABDA,6591
50440,2020-01-02,1530.00,1530.00,1530.00,1530.00,0,ABMM,6591
50441,2020-01-02,1500.00,1535.00,1500.00,1510.00,13727900,ACES,6591
...,...,...,...,...,...,...,...,...
456815,2021-12-30,840.00,860.00,800.00,800.00,20900,YPAS,7079
456816,2021-12-30,1410.00,1560.00,1360.00,1510.00,117600,YULE,7079
456817,2021-12-30,650.00,655.00,625.00,645.00,1667500,ZBRA,7079
456818,2021-12-30,105.00,107.00,103.00,105.00,933951600,ZINC,7079


In [2]:
# tickers = sorted(df["tic"].unique())[:10]
# tickers

In [3]:
# df = df[df["tic"].isin(tickers)]

In [4]:
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] > '2020-01-01']

In [6]:
import numpy as np
from finrl.config import INDICATORS
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.main import check_and_make_directories
import os



# Convert 'date' to datetime
df['date'] = pd.to_datetime(df['date'])

# Feature engineering
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30",
    "dx_30"
]



2024-11-22 17:47:41.228888: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-22 17:47:41.250000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732272461.273638  646593 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732272461.280753  646593 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 17:47:41.309552: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [7]:
df

,date,open,high,low,close,volume,tic,day
50437,2020-01-02,14550.00,14650.00,14025.00,14025.00,799000,AALI,6591
50438,2020-01-02,84.22,85.81,83.43,83.43,7980487,ABBA,6591
50439,2020-01-02,7075.00,7075.00,6925.00,6925.00,1700,ABDA,6591
50440,2020-01-02,1530.00,1530.00,1530.00,1530.00,0,ABMM,6591
50441,2020-01-02,1500.00,1535.00,1500.00,1510.00,13727900,ACES,6591
...,...,...,...,...,...,...,...,...
456815,2021-12-30,840.00,860.00,800.00,800.00,20900,YPAS,7079
456816,2021-12-30,1410.00,1560.00,1360.00,1510.00,117600,YULE,7079
456817,2021-12-30,650.00,655.00,625.00,645.00,1667500,ZBRA,7079
456818,2021-12-30,105.00,107.00,103.00,105.00,933951600,ZINC,7079


In [6]:
def clean_data(data):
        """
        clean the raw data
        deal with missing values
        reasons: stocks could be delisted, not incorporated at the time step
        :param data: (df) pandas dataframe
        :return: (df) pandas dataframe
        """
        df = data.copy()
        df = df.sort_values(["date", "tic"], ignore_index=True)
        df.index = df.date.factorize()[0]
        print(f"DF INDEX: {df.index}")
        merged_closes = df.pivot_table(index="date", columns="tic", values="close")
        print(f"MERGED CLOSES: {merged_closes}")
    
        merged_closes = merged_closes.dropna(axis=1)
        tics = merged_closes.columns
        df = df[df.tic.isin(tics)]
        return df
clean_df = clean_data(df)
clean_df

DF INDEX: Index([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
       ...
       501, 501, 501, 501, 501, 501, 501, 501, 501, 501],
      dtype='int64', length=5020)
MERGED CLOSES: tic            AALI    ABBA    ABDA    ABMM    ACES    ACST    ADES    ADHI  \
date                                                                          
2019-12-10  13500.0   85.81  6975.0  1490.0  1545.0  578.04  1040.0  1230.0   
2019-12-11  13325.0   88.19  6975.0  1490.0  1630.0  595.04  1050.0  1230.0   
2019-12-12  13250.0   87.40  6975.0  1490.0  1590.0  595.04  1040.0  1225.0   
2019-12-13  13425.0   85.81  6975.0  1490.0  1550.0  583.70  1055.0  1225.0   
2019-12-16  13400.0   87.40  6975.0  1490.0  1550.0  592.20  1040.0  1215.0   
...             ...     ...     ...     ...     ...     ...     ...     ...   
2021-12-24   9550.0  313.05  6125.0  1420.0  1320.0  210.00  3200.0   900.0   
2021-12-27   9600.0  328.94  6125.0  1455.0  1285.0  216.00  3300.0   960.0   
2021-12-28   9600.0  306.

,date,open,high,low,close,volume,tic,day
0,2019-12-10,13675.00,13750.0,13100.00,13500.00,1797100,AALI,6578
0,2019-12-10,85.02,87.4,84.22,85.81,52099962,ABBA,6578
0,2019-12-10,6975.00,6975.0,6975.00,6975.00,0,ABDA,6578
0,2019-12-10,1490.00,1490.0,1490.00,1490.00,200,ABMM,6578
0,2019-12-10,1485.00,1560.0,1485.00,1545.00,47184000,ACES,6578
...,...,...,...,...,...,...,...,...
501,2021-12-30,216.00,216.0,208.00,210.00,9038000,ACST,7079
501,2021-12-30,3330.00,3340.0,3250.00,3290.00,104700,ADES,7079
501,2021-12-30,910.00,920.0,895.00,895.00,6074400,ADHI,7079
501,2021-12-30,7700.00,7750.0,7700.00,7700.00,53800,ADMF,7079


In [5]:
fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_turbulence=True,
    user_defined_feature=False
)
df = fe.preprocess_data(df)

# Split the data
train = df[df['date'] < '2022-12-01']
trade = df[df['date'] >= '2022-12-01']

BEFORE CLEAN:              date    open    high     low   close     volume     tic   day
0      2010-02-16    22.0    35.0    19.0    33.0    7181000  KBRI-W  4183
1      2010-02-16   255.0   255.0   255.0   255.0          0  TBLA-W  4183
2      2010-02-17    33.0    33.0    25.0    26.0     424000  KBRI-W  4184
3      2010-02-17   255.0   255.0   255.0   255.0          0  TBLA-W  4184
4      2010-02-18    25.0    40.0    24.0    33.0    6675500  KBRI-W  4185
...           ...     ...     ...     ...     ...        ...     ...   ...
456815 2021-12-30   840.0   860.0   800.0   800.0      20900    YPAS  7079
456816 2021-12-30  1410.0  1560.0  1360.0  1510.0     117600    YULE  7079
456817 2021-12-30   650.0   655.0   625.0   645.0    1667500    ZBRA  7079
456818 2021-12-30   105.0   107.0   103.0   105.0  933951600    ZINC  7079
456819 2021-12-30   410.0   414.0   410.0   414.0     196600    ZONE  7079

[456820 rows x 8 columns]

AFTER CLEAN: Empty DataFrame
Columns: [date, open, high, l

0it [00:00, ?it/s]


KeyError: "None of [Index(['tic', 'date', 'macd'], dtype='object')] are in the [columns]"

In [7]:
indicator_df

NameError: name 'indicator_df' is not defined

In [ ]:
# Create an environment for training
stock_dimension = len(train.tic.unique())
state_space = 1 + 2 * stock_dimension + len(INDICATORS) * stock_dimension
print(f"State space: {state_space}, Stock dimension: {stock_dimension}")

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "reward_scaling": 1e-4,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "mode": "train",
    "data": train,
    "print_verbosity": 5,
    "cash_penalty_proportion": 0.1,
}
e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [ ]:
# Initialize DRLAgent and Train
agent = DRLAgent(env=e_train_gym)
model_ensemble = agent.get_model("a2c")  # Replace "a2c" with the desired ensemble model
trained_model = agent.train_model(model=model_ensemble, tb_log_name="a2c", total_timesteps=50000)

# Prediction
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_model, environment=e_trade_gym
)

In [ ]:
# Save the trained model
model_save_path = "trained_ensemble_model"
trained_model.save(model_save_path)

# Example: Load the model and predict using the latest 5 days of data
from finrl.env.env_stock_trading import StockTradingEnv

# Load the model
loaded_model = DRLAgent.load_model(model_save_path)

# Prepare the latest 5 days of data for prediction
latest_5_days = df[df['date'] >= (df['date'].max() - pd.Timedelta(days=5))]


In [ ]:
latest_env_kwargs = env_kwargs.copy()
latest_env_kwargs['data'] = latest_5_days

e_latest_gym = StockTradingEnv(df=latest_5_days, **latest_env_kwargs)

# Predict action (sell, hold, buy)
df_latest_account_value, df_latest_actions = DRLAgent.DRL_prediction(
    model=loaded_model, environment=e_latest_gym
)

print("Latest actions (sell, hold, buy):")
print(df_latest_actions)